In [6]:
# Import the required libraries and dependencies
import os
import pandas as pd
import plotly.express as px
import hvplot.pandas
from pathlib import Path
from dotenv import load_dotenv
import sqlalchemy

## Clean data

In [7]:
my_file = Path('./Resources/kickstarter_data_clean/ks-projects-large.csv')
if not my_file.is_file():
    display(my_file.is_file())
    %run ./data_clean.ipynb

True

## Enable Mapbox API access token

In [8]:
# Load the .env file into the notebook
load_dotenv()

# Read in your MAPBOX_API_KEY
mapbox_api_access_token = os.getenv('MAPBOX_API_ACCESS_TOKEN')

# Confirm the availability of your Mapbox API access token by checking its type
display(type(mapbox_api_access_token))

# Set your Mapbox API access token
px.set_mapbox_access_token(mapbox_api_access_token)

str

## Add SQL database and simple functions to load and access tables in that database

In [9]:
# Establishes Database Connection with a temporary SQL db (we can update to give it a name later)
database_connection_string = "sqlite:///"

engine = sqlalchemy.create_engine(database_connection_string)

# Function to load table into DB
# data is the dataframe we want to save, 
# table name is the name of the new table (as a string value), 
# and engine is the engine input established earlier
def new_table(data, table_name):
    data.to_sql(f"{table_name}", engine, index=True, if_exists="replace")


# Lets us load the table of our choice from the database, just set the function equal to a new dataframe variable and run 
# must set the table name as a string value
def load_full_table(table_name):
    new_df = pd.read_sql_table(f"{table_name}", con=engine )
    return new_df

new_table(kickstarter_selected_cols_large_recent_df, "kickstarter_large")
new_table(kickstarter_selected_cols_small_most_backed_df, "kickstarter_small")
new_table(indiegogo_selected_cols_df, "indie_gogo")